In [ ]:
import sys

import tensorflow as tf
import tf_utils
from tensorflow_addons.metrics import CohenKappa
from tensorflow.keras.models import Model, load_model
from utils import *
from padding_generator import generator1
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, BatchNormalization, Activation, add, GlobalAveragePooling2D, Dense, ZeroPadding2D, GaussianNoise

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD, Nadam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from res_gen import ResBlockGen

import math

In [ ]:
MODEL_PATHS = data_paths('resnet_34')

input_shape = (224, 224)

bn_momentum = 0.1
initializer = 'he_uniform'

block = ResBlockGen(bn_momentum, initializer).full_preactivation_res_block

#mirrored strategy for distributed training
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    input_img = Input(shape=(*input_shape, 1), name='input')
    
    x = ZeroPadding2D(3, name='zero_padding_1')(input_img)
    x = Conv2D(filters=64, kernel_size=7, strides=2, name='conv_first', kernel_initializer=initializer)(x)
    x = BatchNormalization(name='bn_first', momentum=bn_momentum)(x)
    x = Activation('relu', name='activation_first')(x)

    x = ZeroPadding2D(1, name='zero_padding_2')(x)
    x = MaxPool2D(3, strides=2, name='max_pooling')(x)

    x = block(x, 64, 1, True)
    x = block(x, 64, 1)
    x = block(x, 64, 1)
    x = block(x, 128, 2)
    x = block(x, 128, 1)
    x = block(x, 128, 1)
    x = block(x, 128, 1)
    x = block(x, 256, 2)
    for i in range(5):
        x = block(x, 256, 1)
    x = block(x, 512, 2)
    x = block(x, 512, 1)
    x = block(x, 512, 1)

    x = Activation('relu', name='activation_last')(x)
    x = GlobalAveragePooling2D(name='GAP')(x)
    output = Dense(1, activation='sigmoid', name='output')(x)

    model = Model(inputs=[input_img], outputs=[output])

    optimizer = Nadam(learning_rate=1e-4)
    #optimizer = SGD(learning_rate=0.1, momentum=0.9)

    model.compile(
        optimizer=optimizer,
        metrics=[CohenKappa(num_classes=2), 'accuracy'],
        loss='binary_crossentropy',
    )

In [ ]:
model.summary()

In [ ]:
set_note(MODEL_PATHS, 
        'Resnet-34, input shape 224, aug 1, Nadam 1e-4, bn_m 0.1, full preactivation, batch size 64, CW, mirrored',
         summary_append=model
        )

model.save(MODEL_PATHS['init'])
print('Created: ' + MODEL_PATHS['init'])

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, show_shapes=True)